In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error, precision_score, roc_auc_score
from sklearn import metrics
from xgboost.sklearn import XGBClassifier
import scipy.sparse as sp
from torch.optim import Adam

from layers import SinkhornDistance
from model_peryton2 import *
from utils import *
from get_sim import *
import args
import os
import time
import matplotlib.pyplot as plt
import random
## peryton_sign1  without wloss and aux, crossentropy -> MSE

/home/21031211508/anaconda3/envs/torch_zhu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/21031211508/anaconda3/envs/torch_zhu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
torch.manual_seed(1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# paprameters

random.seed(1)
k1 = 140  # mic
k2 = 5  # dis
A = pd.read_csv('./peryton/sign_final_ass.csv', index_col=0).to_numpy()

print("the number of microbes and diseases", A.shape)
print("the number of associations", sum(sum(A)))
x, y = A.shape
score_matrix = np.zeros([x, y])
samples = get_all_the_samples(A)

# sim_m, sim_d = get_sim_3ss_1ms_peryton(A, k1, k2)
sim_m, sim_d = np.load('./peryton/sim_m_sign.npy'), np.load('./peryton/sim_d_sign.npy')
sim_m_0 = set_digo_zero(sim_m, 0)
sim_d_0 = set_digo_zero(sim_d, 0)

features_m, features_d = get_features_A(A)
features_m, features_d = sparse_to_tuple(sp.coo_matrix(features_m)), sparse_to_tuple(sp.coo_matrix(features_d))
# features_m = sparse_to_tuple(sp.coo_matrix(A))
# features_d = sparse_to_tuple(sp.coo_matrix(A.transpose()))

the number of microbes and diseases (1396, 43)
the number of associations -88.0


In [3]:

adj_norm = preprocess_graph(sim_m_0)

adj = sim_m_0
# pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
# norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
pos_weight = 25
norm = 0.5

sim_m_0 = sp.coo_matrix(sim_m_0)
sim_m_0.eliminate_zeros()
adj_label = sim_m_0 + sp.eye(sim_m_0.shape[0])
adj_label = sparse_to_tuple(adj_label)

adj_norm = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].T),
                                    torch.FloatTensor(adj_norm[1]),
                                    torch.Size(adj_norm[2])).to(device)
adj_label = torch.sparse.FloatTensor(torch.LongTensor(adj_label[0].T),
                                     torch.FloatTensor(adj_label[1]),
                                     torch.Size(adj_label[2])).to_dense().to(device)
features = torch.sparse.FloatTensor(torch.LongTensor(features_m[0].T),
                                    torch.FloatTensor(features_m[1]),
                                    torch.Size(features_m[2])).to(device)

weight_mask = adj_label.view(-1) == 1
weight_tensor = torch.ones(weight_mask.size(0)).to(device)
# weight_tensor[weight_mask] = pos_weight
weight_tensor[weight_mask] = 25

# init model and optimizer
model = VGAE3_2()
model.to(device)
print(model)

# optimizer = Adam(model.parameters(), lr=args.learning_rate)
optimizer = Adam(model.parameters(), lr=args.learning_rate, weight_decay=1e-4)
print("Optimizer", optimizer)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 500, gamma=0.8, last_epoch=-1)

sinkhorn = SinkhornDistance(eps=0.1, max_iter=100, device=device)

tra_auc, tra_ap, tra_l, v_auc, v_ap = [], [], [], [], []
tra_acc, tra_baseloss, tra_aux1, tra_aux2 = [], [], [], []
train_wloss, tra_kl = [], []
tra_r2, tra_RMSE = [], []

min_loss = 10.
min_mse = 100.

VGAE3_2(
  (conv1): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=86, out_features=256, bias=False)
  )
  (conv2): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=256, out_features=128, bias=False)
  )
  (meanGCN1): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=128, out_features=64, bias=False)
  )
  (logstdGCN1): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=128, out_features=64, bias=False)
  )
  (meanGCN2): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=128, out_features=32, bias=False)
  )
  (logstdGCN2): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=128, out_features=32, bias=False)
  )
  (meanGCN3): GraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
    (w): Linear(in_features=128, out_features=16, bias=False)
  )
  (logstdGCN3): GraphConv(
    (dropout)

In [4]:

suffix = 'peryton_sign3'
for epoch in range(20):
    t = time.time()
    model.train()
    A_pred, x1, x2, z = model(adj_norm, features)

    optimizer.zero_grad()
    # loss = base_loss = norm * F.binary_cross_entropy(A_pred.view(-1), adj_label.view(-1), weight=weight_tensor)
    loss = base_loss = norm * F.mse_loss(A_pred.view(-1), adj_label.view(-1))
    tra_baseloss.append(loss.item())

    kl_divergence = 0.5 / A_pred.size(0) * (
            1 + 2 * model.logstd - model.mean ** 2 - torch.exp(model.logstd) ** 2).sum(1).mean() 
    tra_kl.append(-kl_divergence.item())
    loss -= kl_divergence

    # wasser_loss = 0.5 / A_pred.size(0) * sinkhorn(z, torch.randn_like(z))[0]
    # train_wloss.append(wasser_loss.item())
    # loss += wasser_loss * w_w
    # loss += wasser_loss

    loss.backward()
    optimizer.step()
    scheduler.step()

    r2 = r2_score(adj_label.cpu().detach().numpy().reshape(-1), A_pred.cpu().detach().numpy().reshape(-1))
    RMSE = np.sqrt(mean_squared_error(adj_label.cpu().detach().numpy().reshape(-1), A_pred.cpu().detach().numpy().reshape(-1)))
    tra_r2.append(r2)
    tra_RMSE.append(RMSE)

    tra_l.append(loss.item())
    print('--------------------------------')
    print("Epoch:", '%04d' % (epoch + 1), "base_loss=", "{:.5f}".format(base_loss.item()),
          "train_loss=", "{:.5f}".format(loss.item()),
          "train_r2=", "{:.5f}".format(r2), "train_RMSE=", "{:.5f}".format(RMSE),
          "time=", "{:.5f}".format(time.time() - t))
    if RMSE < min_mse:
        min_mse = RMSE
        state = {'model': model.state_dict(),
                 'epoch': epoch,
                 'min_mse': min_mse,
                 }
        torch.save(state, './sign/models/M_mse_256_128_64_5000_' + suffix + '.pth')

model.eval()
save_dic = {'train_loss': tra_l,
            'train_baseloss': tra_baseloss,
            'train_aux1': tra_aux1,
            'train_aux2': tra_aux2,
            'train_wloss': train_wloss,
            'train_kl': tra_kl,
            'train_r2': tra_r2,
            'train_RMSE': tra_RMSE,
            }
np.save('./sign/metrics/M_mse_256_128_64_5000_' + suffix + '.npy', save_dic)
model_ = VGAE3_2()
model_.to(device)
model_.load_state_dict(torch.load('./sign/models/M_mse_256_128_64_5000_' + suffix + '.pth')['model'])
model_.eval()
out_m = model_(adj_norm, features)

latent_m = out_m[-1].cpu().detach().numpy()
np.save('./sign/latent/latent_m_' + suffix + '.npy', latent_m)

--------------------------------
Epoch: 0001 base_loss= 18.34065 train_loss= 18.34065 train_r2= -730.63384 train_RMSE= 6.05650 time= 2.15425
--------------------------------
Epoch: 0002 base_loss= 18.13314 train_loss= 18.13314 train_r2= -722.35582 train_RMSE= 6.02214 time= 2.09686
--------------------------------
Epoch: 0003 base_loss= 17.26904 train_loss= 17.26904 train_r2= -687.88540 train_RMSE= 5.87690 time= 1.95547
--------------------------------
Epoch: 0004 base_loss= 16.49404 train_loss= 16.49404 train_r2= -656.96882 train_RMSE= 5.74351 time= 2.23896
--------------------------------
Epoch: 0005 base_loss= 15.87214 train_loss= 15.87214 train_r2= -632.15958 train_RMSE= 5.63419 time= 2.55503
--------------------------------
Epoch: 0006 base_loss= 14.93691 train_loss= 14.93691 train_r2= -594.85052 train_RMSE= 5.46567 time= 2.08512
--------------------------------
Epoch: 0007 base_loss= 14.42729 train_loss= 14.42729 train_r2= -574.51950 train_RMSE= 5.37161 time= 1.95816
-------------

In [5]:

adj_norm2 = preprocess_graph(sim_d_0)

adj2 = sim_d_0
# pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
# norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
pos_weight2 = 25
norm2 = 0.5

sim_d_0 = sp.coo_matrix(sim_d_0)
sim_d_0.eliminate_zeros()
adj_label2 = sim_d_0 + sp.eye(sim_d_0.shape[0])
adj_label2 = sparse_to_tuple(adj_label2)

adj_norm2 = torch.sparse.FloatTensor(torch.LongTensor(adj_norm2[0].T),
                                     torch.FloatTensor(adj_norm2[1]),
                                     torch.Size(adj_norm2[2])).to(device)
adj_label2 = torch.sparse.FloatTensor(torch.LongTensor(adj_label2[0].T),
                                      torch.FloatTensor(adj_label2[1]),
                                      torch.Size(adj_label2[2])).to_dense().to(device)
features2 = torch.sparse.FloatTensor(torch.LongTensor(features_d[0].T),
                                     torch.FloatTensor(features_d[1]),
                                     torch.Size(features_d[2])).to(device)

weight_mask2 = adj_label2.view(-1) == 1
weight_tensor2 = torch.ones(weight_mask2.size(0)).to(device)
# weight_tensor[weight_mask] = pos_weight
weight_tensor2[weight_mask2] = 25

# init model and optimizer
model2 = VGAE4_2()
model2.to(device)

# optimizer2 = Adam(model2.parameters(), lr=0.001)
optimizer2 = Adam(model2.parameters(), lr=0.001, weight_decay=1e-4)
print("Optimizer", optimizer2)
scheduler2 = torch.optim.lr_scheduler.StepLR(optimizer2, 500, gamma=0.8, last_epoch=-1)

sinkhorn2 = SinkhornDistance(eps=0.1, max_iter=100, device=device)

tra_auc, tra_ap, tra_l, v_auc, v_ap = [], [], [], [], []
tra_acc, tra_baseloss, tra_aux1, tra_aux2 = [], [], [], []
train_wloss, tra_kl = [], []
tra_r2, tra_RMSE = [], []

max_r2 = 0.
min_loss = 10.
min_mse = 100.


Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.0001
)


In [7]:

for epoch in range(20):
    t = time.time()
    model2.train()
    A_pred, x1, x2, z = model2(adj_norm2, features2)

    optimizer2.zero_grad()

    # loss = base_loss = norm2 * F.binary_cross_entropy(A_pred.view(-1), adj_label2.view(-1), weight=weight_tensor2)
    loss = base_loss = norm * F.mse_loss(A_pred.view(-1), adj_label2.view(-1))
    # loss = base_loss = norm2 * torch.sqrt(F.mse_loss(A_pred.view(-1), adj_label2.view(-1)))
    # print('#######################')
    # print('base_loss:', loss.item())
    tra_baseloss.append(loss.item())

    kl_divergence = 0.5 / A_pred.size(0) * (
            1 + 2 * model2.logstd - model2.mean ** 2 - torch.exp(model2.logstd) ** 2).sum(1).mean()
    tra_kl.append(-kl_divergence.item())
    # loss -= kl_divergence * w_w
    loss -= kl_divergence

    # wasser_loss = 0.5 / A_pred.size(0) * sinkhorn2(z, torch.randn_like(z))[0]
    # train_wloss.append(wasser_loss.item())
    # loss += wasser_loss * w_w
    # loss += wasser_loss

    loss.backward()
    optimizer2.step()
    scheduler2.step()

    r2 = r2_score(adj_label2.cpu().detach().numpy().reshape(-1), A_pred.cpu().detach().numpy().reshape(-1))
    RMSE = np.sqrt(mean_squared_error(adj_label2.cpu().detach().numpy().reshape(-1), A_pred.cpu().detach().numpy().reshape(-1)))
    tra_r2.append(r2)
    tra_RMSE.append(RMSE)

    tra_l.append(loss.item())
    print('--------------------------------')
    print("Epoch:", '%04d' % (epoch + 1), "base_loss=", "{:.5f}".format(base_loss.item()),
          "train_loss=", "{:.5f}".format(loss.item()),
          "train_r2=", "{:.5f}".format(r2), "train_RMSE=", "{:.5f}".format(RMSE),
          "time=", "{:.5f}".format(time.time() - t))
    if base_loss.item() < min_loss:
        min_loss = base_loss.item()
        state = {'model': model2.state_dict(),
                 'epoch': epoch,
                 'min_loss': min_loss,
                 }
        torch.save(state, './sign/models/D_loss_512_256_128_5000_' + suffix + '.pth')
model2.eval()

save_dic = {'train_loss': tra_l,
            'train_baseloss': tra_baseloss,
            'train_aux1': tra_aux1,
            'train_aux2': tra_aux2,
            'train_wloss': train_wloss,
            'train_kl': tra_kl,
            'train_r2': tra_r2,
            'train_RMSE': tra_RMSE,
            }
np.save('./sign/metrics/D_loss_512_256_128_5000_' + suffix + '.npy', save_dic)

model2_ = VGAE4_2()
model2_.to(device)
model2_.load_state_dict(torch.load('./sign/models/D_loss_512_256_128_5000_' + suffix + '.pth')['model'])
model2_.eval()
out_d = model2_(adj_norm2, features2)

latent_d = out_d[-1].cpu().detach().numpy()
np.save('./sign/latent/latent_d_' + suffix + '.npy', latent_d)



--------------------------------
Epoch: 0001 base_loss= 39.31962 train_loss= 39.31962 train_r2= -3597.46452 train_RMSE= 8.86212 time= 0.03344
--------------------------------
Epoch: 0002 base_loss= 38.49071 train_loss= 38.49071 train_r2= -3518.96230 train_RMSE= 8.76492 time= 0.02387
--------------------------------
Epoch: 0003 base_loss= 33.06227 train_loss= 33.06227 train_r2= -3012.49852 train_RMSE= 8.10988 time= 0.02259
--------------------------------
Epoch: 0004 base_loss= 29.55837 train_loss= 29.55837 train_r2= -2677.92637 train_RMSE= 7.64644 time= 0.02306
--------------------------------
Epoch: 0005 base_loss= 25.55394 train_loss= 25.55394 train_r2= -2292.49586 train_RMSE= 7.07503 time= 0.02138
--------------------------------
Epoch: 0006 base_loss= 21.46865 train_loss= 21.46865 train_r2= -1889.92786 train_RMSE= 6.42416 time= 0.05483
--------------------------------
Epoch: 0007 base_loss= 20.63111 train_loss= 20.63111 train_r2= -1777.17865 train_RMSE= 6.22969 time= 0.05073
------

: 